In [54]:
import sqlite3
import pandas as pd
from sqlite3 import Error

In [59]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# To execute queries in SQLite, use cursor.execute().
def ex_q(connection, query): # execute query
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def er_q(connection, query): #execute read query
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")
# OMG IT MAKES SUCH A BEAUTFIFUL DIFFERENCE!!!! 


def display_query(query,connection=connection):
    q = er_q(query,connection)
    return pd.read_sql_query(q, connection)


In [7]:
connection = create_connection("northwind.db") # in this case the database is in the same

Connection to SQLite DB successful


In [9]:
drop_end_obs_dates_table = '''DROP TABLE if exists end_obs_dates;'''


In [12]:
create_end_obs_dates_table = '''CREATE TABLE end_obs_dates 
AS

-- Recursive query example (something like FOR loop for SQL)
WITH RECURSIVE
  cnt(x) AS (
     SELECT 0
     UNION ALL
     SELECT x+1 FROM cnt
      LIMIT (SELECT ROUND(((julianday('1998-06-01') - julianday('1996-08-01'))/30) + 1)) -- this counts number of months between these two days
  )
SELECT date('1996-08-01', '+' || x || ' month') as end_obs_date FROM cnt;'''


In [13]:
ex_q(connection,drop_end_obs_dates_table)
ex_q(connection,create_end_obs_dates_table)

Query executed successfully
Query executed successfully


In [15]:
drop_ads_population_hist = '''DROP TABLE if exists ads_population_hist;'''


In [16]:
create_ads_population_hist = '''
CREATE TABLE ads_population_hist 
AS
SELECT A.*,
       B.*
FROM end_obs_dates AS A
CROSS JOIN (SELECT DISTINCT customerid FROM customers) AS B
;
'''

In [17]:
ex_q(connection,drop_ads_population_hist)
ex_q(connection,create_ads_population_hist)

Query executed successfully
Query executed successfully


With Queries it would important to add a header for the columns (from the select query)

In [33]:
q1 = """select * from ads_population_hist limit 10;
"""
er_q(connection,q1)

[('1996-08-01', 'ALFKI'),
 ('1996-08-01', 'ANATR'),
 ('1996-08-01', 'ANTON'),
 ('1996-08-01', 'AROUT'),
 ('1996-08-01', 'BERGS'),
 ('1996-08-01', 'BLAUS'),
 ('1996-08-01', 'BLONP'),
 ('1996-08-01', 'BOLID'),
 ('1996-08-01', 'BONAP'),
 ('1996-08-01', 'BOTTM')]

Now, we have the main table where every customer has exactly one row for every month, even when they didn't buy anything.

Our main goal is to create a table where we keep all the important information about our clients. In this tutorial, we will work with clients' orders and create following variables:

   * noofitems
   * noofdistinct_orders
   * total_price


In [34]:
q2 = '''SELECT *,
       unitprice*quantity AS totalprice_for_product
FROM "Order Details" LIMIT 20
'''
er_q(connection,q2)

[(10248, 11, 14.0, 12, 0.0, 168.0),
 (10248, 42, 9.8, 10, 0.0, 98.0),
 (10248, 72, 34.8, 5, 0.0, 174.0),
 (10249, 14, 18.6, 9, 0.0, 167.4),
 (10249, 51, 42.4, 40, 0.0, 1696.0),
 (10250, 41, 7.7, 10, 0.0, 77.0),
 (10250, 51, 42.4, 35, 0.15, 1484.0),
 (10250, 65, 16.8, 15, 0.15, 252.0),
 (10251, 22, 16.8, 6, 0.05, 100.80000000000001),
 (10251, 57, 15.6, 15, 0.05, 234.0),
 (10251, 65, 16.8, 20, 0.0, 336.0),
 (10252, 20, 64.8, 40, 0.05, 2592.0),
 (10252, 33, 2.0, 25, 0.05, 50.0),
 (10252, 60, 27.2, 40, 0.0, 1088.0),
 (10253, 31, 10.0, 20, 0.0, 200.0),
 (10253, 39, 14.4, 42, 0.0, 604.8000000000001),
 (10253, 49, 16.0, 40, 0.0, 640.0),
 (10254, 24, 3.6, 15, 0.15, 54.0),
 (10254, 55, 19.2, 21, 0.15, 403.2),
 (10254, 74, 8.0, 21, 0.0, 168.0)]

In [35]:
q3 = '''SELECT A.orderid,
     COUNT(DISTINCT A.productid) AS no_of_distinct_products,
     SUM(A.quantity) AS no_of_items,
     SUM(A.totalprice_for_product) AS total_price 
     FROM (SELECT *,
                  unitprice*quantity AS totalprice_for_product
           FROM "Order Details") AS A
GROUP BY 1
LIMIT 20;
'''
er_q(connection,q3)

[(10248, 3, 27, 440.0),
 (10249, 2, 49, 1863.4),
 (10250, 3, 60, 1813.0),
 (10251, 3, 41, 670.8),
 (10252, 3, 105, 3730.0),
 (10253, 3, 102, 1444.8000000000002),
 (10254, 3, 57, 625.2),
 (10255, 4, 110, 2490.5),
 (10256, 2, 27, 517.8),
 (10257, 3, 46, 1119.9),
 (10258, 3, 121, 2018.6),
 (10259, 2, 11, 100.8),
 (10260, 4, 102, 1746.2),
 (10261, 2, 40, 448.0),
 (10262, 3, 29, 624.8),
 (10263, 4, 184, 2464.8),
 (10264, 2, 60, 724.5),
 (10265, 2, 50, 1176.0),
 (10266, 1, 12, 364.79999999999995),
 (10267, 3, 135, 4031.0)]

In [36]:
q4 = '''SELECT 
    orderid,
    customerid,
    orderdate
FROM orders 
LIMIT 10;
'''
er_q(connection,q4)

[(10248, 'VINET', '1996-07-04'),
 (10249, 'TOMSP', '1996-07-05'),
 (10250, 'HANAR', '1996-07-08'),
 (10251, 'VICTE', '1996-07-08'),
 (10252, 'SUPRD', '1996-07-09'),
 (10253, 'HANAR', '1996-07-10'),
 (10254, 'CHOPS', '1996-07-11'),
 (10255, 'RICSU', '1996-07-12'),
 (10256, 'WELLI', '1996-07-15'),
 (10257, 'HILAA', '1996-07-16')]

We need to assign end_obs_date to each order.

In [31]:
q5 = '''SELECT orderid,
       customerid,
       orderdate,
       date(orderdate,'start of month','+1 month') as end_obs_date
FROM orders LIMIT 10;
'''
er_q(connection,q5)

[(10248, 'VINET', '1996-07-04', '1996-08-01'),
 (10249, 'TOMSP', '1996-07-05', '1996-08-01'),
 (10250, 'HANAR', '1996-07-08', '1996-08-01'),
 (10251, 'VICTE', '1996-07-08', '1996-08-01'),
 (10252, 'SUPRD', '1996-07-09', '1996-08-01'),
 (10253, 'HANAR', '1996-07-10', '1996-08-01'),
 (10254, 'CHOPS', '1996-07-11', '1996-08-01'),
 (10255, 'RICSU', '1996-07-12', '1996-08-01'),
 (10256, 'WELLI', '1996-07-15', '1996-08-01'),
 (10257, 'HILAA', '1996-07-16', '1996-08-01')]

In [38]:
drop_ads_orders_hist = '''DROP TABLE if exists ads_orders_hist;'''
ex_q(connection,drop_ads_orders_hist)

Query executed successfully


In [41]:
create_ads_orders_hist = '''CREATE TABLE ads_orders_hist 
AS
SELECT A.orderid,
       A.customerid,
       A.end_obs_date,
       B.no_of_distinct_products,
       B.no_of_items,
       B.total_price
FROM (
    SELECT orderid,
             customerid,
             orderdate,
             date(orderdate,'start of month','+1 month') as end_obs_date
    FROM orders)
AS A
LEFT OUTER JOIN (
    SELECT A.orderid,
         COUNT(DISTINCT A.productid) AS no_of_distinct_products,
         SUM(A.quantity) AS no_of_items,
         SUM(A.totalprice_for_product) AS total_price
    FROM (
        SELECT *,
            unitprice*quantity AS totalprice_for_product
        FROM "Order Details") 
    AS A
    GROUP BY 1) 
AS B 
ON A.orderid = B.orderid;
'''
ex_q(connection,create_ads_orders_hist)

Query executed successfully


In [44]:
q6 = '''select orderid
    ,count(*)
from ads_orders_hist
group by 1
order by 2 desc
limit 5
;
'''
er_q(connection,q6)

[(11077, 1), (11076, 1), (11075, 1), (11074, 1), (11073, 1)]

In [45]:
drop_ads_observation_hist = '''DROP table if exists ads_observation_hist;'''
ex_q(connection,drop_ads_observation_hist)

Query executed successfully


In [47]:
create_ads_observation_hist = '''create table ads_observation_hist as
select 
    A.*
    -- we can replace missings with 0 because it means there were no orders for this client during specific month.
    ,coalesce(B.no_of_distinct_orders_1M, 0) as no_of_distinct_orders_1M
    ,coalesce(B.no_of_items_1M, 0) as no_of_items_1M
    ,coalesce(B.total_price_1M, 0) as total_price_1M
from ads_population_hist as A
left outer join (
    -- we need to group by our orders to customer level
    select customerid
        ,end_obs_date
        ,count(distinct orderid) as no_of_distinct_orders_1M
        ,sum(no_of_items) as no_of_items_1M
        ,sum(total_price) as total_price_1M
    from ads_orders_hist
    group by 1,2
) as B
on A.customerid = B.customerid
  and A.end_obs_date = B.end_obs_date
;'''
ex_q(connection,create_ads_observation_hist)

Query executed successfully


In [48]:
q7 = '''select customerid
    ,end_obs_date
    ,count(*)
from ads_observation_hist
group by 1,2
order by 3 desc
limit 5
;
'''
er_q(connection,q7)

[('ALFKI', '1996-08-01', 1),
 ('ALFKI', '1996-09-01', 1),
 ('ALFKI', '1996-10-01', 1),
 ('ALFKI', '1996-11-01', 1),
 ('ALFKI', '1996-12-01', 1)]

In [82]:
q8 = '''select * from ads_observation_hist WHERE customerid = 'BOTTM' ORDER BY customerid limit 50;
'''
#er_q(connection,q8)
display_query(q8)

,end_obs_date,customerid,no_of_distinct_orders_1M,no_of_items_1M,total_price_1M
0,1996-08-01,BOTTM,0,0,0.0
1,1996-09-01,BOTTM,0,0,0.0
2,1996-10-01,BOTTM,0,0,0.0
3,1996-11-01,BOTTM,0,0,0.0
4,1996-12-01,BOTTM,0,0,0.0
5,1997-01-01,BOTTM,1,81,1832.8
6,1997-02-01,BOTTM,3,269,4533.5
7,1997-03-01,BOTTM,0,0,0.0
8,1997-04-01,BOTTM,0,0,0.0
9,1997-05-01,BOTTM,1,80,896.0




Create following attributes in table ads_observation_hist using SQLWorkbench and database Northwind.

    * noofitems_3M (use window functions)
    * totalprice3M (use window functions)
    * maxmonthlytotalprice3M (use window functions)
    * minmonthlytotalprice3M (use window functions)
    * avgnoofitems3M (use window functions)

All attributes should be in the table that is monthly aggregated and every client is there for every month.
Note

You can continue directly after the ADS tutorial and build on top of the table ads_observation_hist.


In [110]:
q9 = '''
SELECT
    end_obs_date,
    customerid,
    no_of_distinct_orders_1M,
    SUM(no_of_distinct_orders_1M) OVER ( ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS no_of_orders_3M,
    AVG(no_of_distinct_orders_1M) OVER ( ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS avg_no_of_items_3M,
    no_of_items_1M,
    SUM(no_of_items_1M) OVER ( ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS no_of_items_3M, 
    total_price_1M,
    SUM(total_price_1M) OVER ( ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS total_price_3M,
    MAX(total_price_1M) OVER (ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as max_monthly_total_price_3M,
    MIN(total_price_1M) OVER (ORDER BY customerid ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as min_monthly_total_price_3M
    --MAX(total_price_1m) OVER (PARTITION BY customerid) AS max_monthly_total_price_3M,
    --MIN(total_price_1m) OVER (PARTITION BY customerid) AS min_monthly_total_price_3M
FROM ads_observation_hist
-- WHERE customerid = 'BOTTM'
-- ORDER BY customerid
--ORDER BY end_obs_date
limit 10 OFFSET 10
'''

#date(end_obs_date,'start of month','+2 month'

display_query(q9)

,end_obs_date,customerid,no_of_distinct_orders_1M,no_of_orders_3M,avg_no_of_items_3M,no_of_items_1M,no_of_items_3M,total_price_1M,total_price_3M,max_monthly_total_price_3M,min_monthly_total_price_3M
0,1997-06-01,ALFKI,0,0,0.000000,0,0,0.0,0.0,0.0,0
1,1997-07-01,ALFKI,0,0,0.000000,0,0,0.0,0.0,0.0,0
2,1997-08-01,ALFKI,0,0,0.000000,0,0,0.0,0.0,0.0,0
3,1997-09-01,ALFKI,1,1,0.333333,38,38,1086.0,1086.0,1086.0,0
4,1997-10-01,ALFKI,0,1,0.333333,0,38,0.0,1086.0,1086.0,0
5,1997-11-01,ALFKI,2,3,1.000000,41,79,1208.0,2294.0,1208.0,0
6,1997-12-01,ALFKI,0,2,0.666667,0,41,0.0,1208.0,1208.0,0
7,1998-01-01,ALFKI,0,2,0.666667,0,41,0.0,1208.0,1208.0,0
8,1998-02-01,ALFKI,1,1,0.333333,17,17,851.0,851.0,851.0,0
9,1998-03-01,ALFKI,0,1,0.333333,0,17,0.0,851.0,851.0,0


In [121]:
alter_ads_observation_hist = '''
ALTER TABLE ads_observation_hist 
    ADD COLUMN no_of_items_3M. 
    ADD COLUMN total_price_3M,
    ADD COLUMN max_monthly_total_price_3M, 
    ADD COLUMN min_monthly_total_price_3M,
    ADD COLUMN avg_no_of_items_3M;'''

'''ALTER TABLE ads_observation_hist 
    ADD COLUMN no_of_items_3m SMALLINT, 
    ADD COLUMN total_price_3M FLOAT6,
    ADD COLUMN max_monthly_total_price_3M FLOAT6, 
    ADD COLUMN min_monthly_total_price_3M FLOAT6,
    ADD COLUMN avg_no_of_items_3M FLOAT6;'''
# SMALLINT


ex_q(connection, alter_ads_observation_hist)

The error 'near ".": syntax error' occurred
